In [8]:
!pip install h2o

     |████████████████████████████████| 177.0 MB 19 kB/s 
  Created wheel for h2o: filename=h2o-3.36.1.1-py2.py3-none-any.whl size=177068062 sha256=b172531a1f64b93faba59e6d73eef93a656c864148f19b4ff8503293974cbe8c
  Stored in directory: /root/.cache/pip/wheels/a6/d9/ab/5442447c7e2ccf07f66aa8b79f3877ce5382f0b95e6c0c797b
Successfully built h2o


In [9]:
import h2o
from h2o.automl import H2OAutoML
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
import sklearn

In [12]:
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.15" 2022-04-19; OpenJDK Runtime Environment (build 11.0.15+10-Ubuntu-0ubuntu0.18.04.1); OpenJDK 64-Bit Server VM (build 11.0.15+10-Ubuntu-0ubuntu0.18.04.1, mixed mode, sharing)
  Starting server from /usr/local/lib/python3.7/dist-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmpxwo1ll7t
  JVM stdout: /tmp/tmpxwo1ll7t/h2o_unknownUser_started_from_python.out
  JVM stderr: /tmp/tmpxwo1ll7t/h2o_unknownUser_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,03 secs
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.36.1.1
H2O_cluster_version_age:,17 days
H2O_cluster_name:,H2O_from_python_unknownUser_7jhdil
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.172 Gb
H2O_cluster_total_cores:,2
H2O_cluster_allowed_cores:,2
H2O_cluster_status:,"locked, healthy"


In [16]:
import pandas as pd
def dataSetup(name):
  X,y = sklearn.datasets.fetch_openml(name, as_frame=True, return_X_y=True)
  train = pd.concat([X, y], axis=1, join='inner')
  train.dropna()
  train = train.apply(lambda x: pd.factorize(x)[0])
  X,y = train.iloc[:,:-1], train.iloc[:, -1]
  # X = X.apply(lambda x: pd.factorize(x)[0])
  X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X,y,random_state=42)
  y_train = y_train.to_frame(name="class")
  y_test = y_test.to_frame(name="class")
  test = pd.concat([X_test, y_test], axis=1, join='inner')
  train = pd.concat([X_train, y_train], axis=1, join='inner')
  test = h2o.H2OFrame(test)
  train = h2o.H2OFrame(train)
  return (test, train)

In [17]:
test, train = dataSetup('adult')

/usr/local/lib/python3.7/dist-packages/sklearn/datasets/_openml.py:421: UserWarning: Multiple active versions of the dataset matching the name adult exist. Versions may be fundamentally different, returning version 1.
  " {version}.".format(name=name, version=res[0]["version"])


Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [21]:
train.shape

(36631, 15)

In [22]:
x = train.columns
y = "class"
x.remove(y)

In [23]:
train[y] = train[y].asfactor()
test[y] = test[y].asfactor()

In [24]:
aml = H2OAutoML(max_runtime_secs = 3600)
aml.train(x=x, y=y, training_frame=train)
# train = h2o.H2OFrame(train)

AutoML progress: |██████████████████████████████████████████████████████████████
22:49:15.473: GBM_lr_annealing_selection_AutoML_1_20220430_214956 [GBM lr_annealing] failed: water.exceptions.H2OIllegalArgumentException: Can only convert jobs producing a single Model or ModelContainer.

█| (done) 100%
Model Details
H2OStackedEnsembleEstimator :  Stacked Ensemble
Model Key:  StackedEnsemble_AllModels_3_AutoML_1_20220430_214956

No model summary for this model

ModelMetricsBinomialGLM: stackedensemble
** Reported on train data. **

MSE: 0.08657759804577127
RMSE: 0.29424071445972816
LogLoss: 0.2724110013525869
Null degrees of freedom: 9833
Residual degrees of freedom: 9813
Null deviance: 10750.078539726452
Residual deviance: 5357.779574602678
AIC: 5399.779574602678
AUC: 0.9332503251759574
AUCPR: 0.827025394097305
Gini: 0.8665006503519148

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.353137631841047: 


,,0,1,Error,Rate
0,0,6640.0,872.0,0.1161,(872.0/7512.0)
1,1,458.0,1864.0,0.1972,(458.0/2322.0)
2,Total,7098.0,2736.0,0.1352,(1330.0/9834.0)



Maximum Metrics: Maximum metrics at their respective thresholds


,metric,threshold,value,idx
0,max f1,0.353138,0.737050,212.0
1,max f2,0.168052,0.817841,286.0
2,max f0point5,0.612725,0.759775,121.0
3,max accuracy,0.478719,0.874720,164.0
4,max precision,0.995484,1.000000,0.0
5,max recall,0.008647,1.000000,386.0
6,max specificity,0.995484,1.000000,0.0
7,max absolute_mcc,0.373046,0.651080,205.0
8,max min_per_class_accuracy,0.296525,0.850991,232.0
9,max mean_per_class_accuracy,0.238522,0.853543,254.0



Gains/Lift Table: Avg response rate: 23.61 %, avg score: 24.11 %


...


Gains/Lift Table: Avg response rate: 23.82 %, avg score: 23.82 %


,group,cumulative_data_fraction,lower_threshold,lift,cumulative_lift,response_rate,score,cumulative_response_rate,cumulative_score,capture_rate,cumulative_capture_rate,gain,cumulative_gain,kolmogorov_smirnov
0,1,0.010019,0.990480,4.198877,4.198877,1.000000,0.993190,1.000000,0.993190,0.042068,0.042068,319.887666,319.887666,0.042068
1,2,0.020010,0.979706,4.198877,4.198877,1.000000,0.985888,1.000000,0.989544,0.041953,0.084021,319.887666,319.887666,0.084021
2,3,0.030002,0.952980,4.141515,4.179773,0.986339,0.967829,0.995450,0.982312,0.041380,0.125401,314.151496,317.977349,0.125222
3,4,0.040021,0.909974,3.947173,4.121544,0.940054,0.933125,0.981583,0.969999,0.039546,0.164947,294.717288,312.154401,0.163980
4,5,0.050012,0.871179,3.785872,4.054483,0.901639,0.890506,0.965611,0.954117,0.037827,0.202774,278.587240,305.448298,0.200516
5,6,0.100025,0.742341,3.291259,3.672871,0.783843,0.801740,0.874727,0.877929,0.164603,0.367377,229.125922,267.287110,0.350930
6,7,0.150010,0.620958,2.765617,3.370563,0.658656,0.680474,0.802730,0.812134,0.138239,0.505617,176.561729,237.056323,0.466773
7,8,0.200022,0.499189,2.372182,3.120934,0.564956,0.560927,0.743278,0.749324,0.118638,0.624255,137.218196,212.093385,0.556852
8,9,0.300019,0.312850,1.697662,2.646553,0.404313,0.398530,0.630300,0.632403,0.169762,0.794017,69.766212,164.655311,0.648426
9,10,0.400016,0.178388,1.043128,2.245724,0.248430,0.244104,0.534839,0.535335,0.104310,0.898326,4.312797,124.572418,0.654087




Cross-Validation Metrics Summary: 


,,mean,sd,cv_1_valid,cv_2_valid,cv_3_valid,cv_4_valid,cv_5_valid
0,accuracy,0.846868,0.006214,0.838113,0.846478,0.847901,0.855599,0.846249
1,auc,0.913905,0.002440,0.911860,0.911877,0.913001,0.917467,0.915321
2,err,0.153132,0.006214,0.161887,0.153522,0.152099,0.144401,0.153751
3,err_count,1121.800000,44.600450,1191.000000,1129.000000,1105.000000,1069.000000,1115.000000
4,f0point5,0.671831,0.013084,0.658343,0.667196,0.668508,0.693517,0.671589
5,f1,0.706232,0.006318,0.705853,0.704218,0.698992,0.716370,0.705727
6,f2,0.744607,0.010329,0.760754,0.745590,0.732390,0.740781,0.743521
7,lift_top_group,4.199857,0.063407,4.130826,4.245958,4.283608,4.156653,4.182238
8,logloss,0.304812,0.003831,0.310211,0.307218,0.303368,0.300784,0.302476
9,max_per_class_error,0.227225,0.018520,0.197642,0.224018,0.243514,0.241999,0.228950



See the whole table with table.as_data_frame()


In [25]:
lb = aml.leaderboard
lb.head(rows=lb.nrows)

model_id,auc,logloss,aucpr,mean_per_class_error,rmse,mse
StackedEnsemble_AllModels_3_AutoML_1_20220430_214956,0.913898,0.304817,0.784196,0.180376,0.31232,0.0975435
StackedEnsemble_AllModels_4_AutoML_1_20220430_214956,0.913875,0.304891,0.7841,0.180108,0.312352,0.0975637
StackedEnsemble_BestOfFamily_4_AutoML_1_20220430_214956,0.91375,0.305143,0.784163,0.179628,0.312325,0.0975466
GBM_grid_1_AutoML_1_20220430_214956_model_14,0.9136,0.306831,0.782992,0.180448,0.312755,0.0978158
GBM_grid_1_AutoML_1_20220430_214956_model_21,0.913053,0.307206,0.781197,0.179164,0.313259,0.0981314
StackedEnsemble_BestOfFamily_5_AutoML_1_20220430_214956,0.912965,0.306662,0.781597,0.179159,0.312847,0.0978732
GBM_grid_1_AutoML_1_20220430_214956_model_13,0.91289,0.308485,0.781919,0.179198,0.313266,0.0981359
StackedEnsemble_AllModels_2_AutoML_1_20220430_214956,0.912669,0.306584,0.782396,0.183403,0.313088,0.0980238
GBM_grid_1_AutoML_1_20220430_214956_model_29,0.912503,0.307902,0.781609,0.177664,0.313533,0.0983032
StackedEnsemble_BestOfFamily_3_AutoML_1_20220430_214956,0.912281,0.307105,0.781296,0.178505,0.313411,0.0982267
